In [22]:
import os
import json
from datetime import datetime
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import box, mapping
import pystac
import xml.etree.ElementTree as ET
import sys
import fiona
import geopandas as gpd
sys.path.append('/home/vishnu/corestack_STAC')
from constants import DEFAULT_START_DATE, DEFAULT_END_DATE



In [21]:
!pip install fiona
!pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.4/338.4 KB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:01:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.6/27.6 MB 4.7 MB/s eta 0:00:00m eta 0:00:010:00:01


In [4]:
input_file = "/home/vishnu/corestack_STAC/data/swb2_saraikela-kharsawan_gobindpur.geojson"
qgis_style_path = "/home/vishnu/corestack_STAC/data/swb_style.qml"
data_dir = os.path.dirname(input_file)


In [5]:
def parse_qml_to_classes(qml_path):
    try:
        tree = ET.parse(qml_path)
        root = tree.getroot()
        classes = []
        for entry in root.findall(".//paletteEntry"):
            value = int(entry.attrib.get("value", -1))
            label = entry.attrib.get("label", f"Class {value}")
            color = entry.attrib.get("color", "#000000")
            classes.append({
                "value": value,
                "name": label,
                "color": color
            })
        return sorted(classes, key=lambda x: x["value"])
    except Exception as e:
        print(f" Failed to parse QML: {e}")
        return []


In [ ]:
with fiona.open(input_file) as src:
    bounds = src.bounds  # (minx, miny, maxx, maxy)
    bbox = [bounds[0], bounds[1], bounds[2], bounds[3]]
    geometry = mapping(box(*bbox))


    crs = src.crs
    epsg = None
    if "init" in crs:
        epsg = int(crs["init"].split(":")[1])
    elif "epsg" in crs:
        epsg = int(crs["epsg"])

print("BBox:", bbox)
print("EPSG:", epsg)


BBox: [85.83186726151115, 22.438299037364732, 86.2031453696048, 22.761334210815875]
EPSG: 4326


In [12]:
filename = os.path.basename(input_file)
parts = filename.split('_')
try:
    start_date = datetime.strptime(parts[2], "%Y-%m-%d")
    end_date = datetime.strptime(parts[3], "%Y-%m-%d")
except Exception as e:
    print(f"Failed to extract dates from filename '{filename}': {e}")
    start_date = DEFAULT_START_DATE
    end_date = DEFAULT_END_DATE


Failed to extract dates from filename 'swb2_saraikela-kharsawan_gobindpur.geojson': time data 'gobindpur.geojson' does not match format '%Y-%m-%d'


In [15]:
output_dir = "/home/vishnu/corestack_STAC/output_catalog_vector"
item_id = "gobindpur-geojson"
item_dir = os.path.join(output_dir, item_id)
os.makedirs(item_dir, exist_ok=True)

catalog = pystac.Catalog(
    id="gobindpur-geojson-catalog",
    description=""
)


if isinstance(start_date, str):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
if isinstance(end_date, str):
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

item = pystac.Item(
    id=item_id,
    geometry=geometry,
    bbox=bbox,
    datetime=start_date,
    properties={
        "start_datetime": start_date.isoformat() + "Z",
        "end_datetime": end_date.isoformat() + "Z",
        "proj:epsg": epsg,
        "proj:bbox": bbox
    },
    stac_extensions=[
        "https://stac-extensions.github.io/projection/v1.0.0/schema.json"
    ]
)



In [ ]:
item.add_asset(
    key="vector-data",
    asset=pystac.Asset(
        href="home/vishnu/corestack_STAC/data/" + os.path.basename(input_file),
        media_type=pystac.MediaType.GEOJSON,
        roles=["data"],
        title="Gobindpur Vector (GEOJSON)"
    )
)


In [ ]:
if os.path.exists(qgis_style_path):
    print("QML file found. Parsing classification...")
    lulc_classes = parse_qml_to_classes(qgis_style_path)

    item.add_asset(
        key="qgis-style",
        asset=pystac.Asset(
            href="home/vishnu/corestack_STAC/data/swb_style.qml",
            media_type="application/xml",
            roles=["style"],
            title="QGIS Style File"
        )
    )

    item.properties["classification:classes"] = lulc_classes

    legend_path = os.path.join(data_dir, "legend.json")
    with open(legend_path, "w") as f:
        json.dump(lulc_classes, f, indent=2)

    item.add_asset(
        key="legend",
        asset=pystac.Asset(
            href="home/vishnu/corestack_STAC/data/legend.json",
            media_type="application/json",
            roles=["legend"],
            title="LULC Legend"
        )
    )
else:
    print("QML file not found. Skipping classification section.")


QML file found. Parsing classification...


In [24]:
gdf = gpd.read_file(input_file)


thumb_path = os.path.join(data_dir, "thumbnail.png")

plt.figure(figsize=(3, 3))
gdf.plot(edgecolor='black', linewidth=0.5)
plt.axis('off')
plt.savefig(thumb_path, bbox_inches='tight', pad_inches=0, dpi=150)
plt.close()

item.add_asset(
    key="thumbnail",
    asset=pystac.Asset(
        href=os.path.relpath(thumb_path, item_dir), 
        media_type="home/vishnu/corestack_STAC/data/thumbnail.png",
        roles=["thumbnail"],
        title="Thumbnail Preview"
    )
)


<Figure size 300x300 with 0 Axes>

In [25]:
catalog.add_item(item)
catalog.normalize_hrefs(output_dir)
catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)


default_item_path = os.path.join(item_dir, "item.json")
custom_item_path = os.path.join(item_dir, f"{item_id}.json")
if os.path.exists(default_item_path):
    os.rename(default_item_path, custom_item_path)

print("\nSTAC catalog created with:")
print(" Dates from filename")
print(" classification:classes from QML")
print(" Thumbnail preview")
print(" catalog.json:", os.path.join(output_dir, "catalog.json"))
print("item:", custom_item_path)



STAC catalog created with:
 Dates from filename
 classification:classes from QML
 Thumbnail preview
 catalog.json: /home/vishnu/corestack_STAC/output_catalog_vector/catalog.json
item: /home/vishnu/corestack_STAC/output_catalog_vector/gobindpur-geojson/gobindpur-geojson.json
